In [1]:
from typing import Any, Callable, Dict, Optional
from numpy import ndarray
from scipy.sparse import csc_matrix
from typing import Union
from cvxopt import matrix, spmatrix
from typing import Optional
from cvxopt.solvers import options, qp
from numpy import array, ndarray
import numpy as np

In [2]:
CvxoptReadyMatrix = Union[ndarray, matrix, spmatrix]
options["show_progress"] = False  # disable cvxopt output


def cvxopt_matrix(M: CvxoptReadyMatrix) -> matrix:
    if isinstance(M, ndarray):
        return matrix(M)
    if isinstance(M, (spmatrix, matrix)):
        return M
    coo = M.tocoo()
    return spmatrix(
        coo.data.tolist(), coo.row.tolist(), coo.col.tolist(), size=M.shape
    )


def cvxopt_solve_qp(
    P: CvxoptReadyMatrix,
    q: CvxoptReadyMatrix,
    G: Optional[CvxoptReadyMatrix] = None,
    h: Optional[CvxoptReadyMatrix] = None,
    A: Optional[CvxoptReadyMatrix] = None,
    b: Optional[CvxoptReadyMatrix] = None,
    solver: str = None,
    initvals: Optional[ndarray] = None,
    verbose: bool = False,
) -> Optional[ndarray]:
   
    options["show_progress"] = verbose
    args = [cvxopt_matrix(P), cvxopt_matrix(q)]
    kwargs = {"G": None, "h": None, "A": None, "b": None}
    if G is not None and h is not None:
        kwargs["G"] = cvxopt_matrix(G)
        kwargs["h"] = cvxopt_matrix(h)
    if A is not None and b is not None:
        kwargs["A"] = cvxopt_matrix(A)
        kwargs["b"] = cvxopt_matrix(b)
    sol = qp(*args, solver=solver, initvals=initvals, **kwargs)
    if "optimal" not in sol["status"]:
        return None
    return array(sol["x"]).reshape((q.shape[0],))

In [3]:
m = 5
n = 700
p = 5

P = np.loadtxt('P.txt')
q = np.loadtxt('q.txt')
G = np.loadtxt('G.txt')
h = np.loadtxt('h.txt')
A = np.loadtxt('A.txt')
b = np.loadtxt('b.txt')

x=cvxopt_solve_qp( P,q, G,h,A,b)
v=0.5*x.T@P@x+q.T@x
print(v)

-42.314082626817374
